<a href="https://colab.research.google.com/github/kouliki/celebal_task-12/blob/main/Celebal_Task_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing **dataset**

In [5]:
! pip install -q kaggle

In [6]:
from google.colab import files

In [4]:
files.upload()



Saving kaggle (6).json to kaggle (6).json


{'kaggle (6).json': b'{"username":"koulikimahato","key":"b6ce180c38dd23a9a62574b9da0bc963"}'}

In [7]:
! mkdir ~/.kaggle

In [8]:
! cp kaggle.json ~/.kaggle/



cp: cannot stat 'kaggle.json': No such file or directory


In [9]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [10]:
! kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [11]:
! kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 89% 23.0M/25.7M [00:01<00:00, 27.2MB/s]
100% 25.7M/25.7M [00:01<00:00, 16.7MB/s]


In [12]:
! unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
df=pd.read_csv('IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [15]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
df.isnull().sum()

,0
review,0
sentiment,0


In [18]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [19]:
df['review'].head()

,review
0,One of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...
2,I thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is..."


In [20]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [22]:
import re

def clean_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

df['clean_review'] = df['review'].apply(clean_text)

### lowercasing

In [23]:
df['clean_review']=df['clean_review'].apply(lambda x:x.lower())

### Removing Stop Words

In [25]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords=set(stopwords.words('english'))

def remove_stopwords(text):
    words=text.split()
    filtered_words=[word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

df['clean_review']=df['clean_review'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
df['clean_review' ]

,clean_review
0,one reviewers mentioned watching oz episode yo...
1,wonderful little production filming technique ...
2,thought wonderful way spend time hot summer we...
3,basically theres family little boy jake thinks...
4,petter matteis love time money visually stunni...
...,...
49995,thought movie right good job wasnt creative or...
49996,bad plot bad dialogue bad acting idiotic direc...
49997,catholic taught parochial elementary schools n...
49998,im going disagree previous comment side maltin...


### Lemmatization

In [27]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
lemmatizer=WordNetLemmatizer()

def lematize_text(text):
    words=text.split()
    lemmatized_words=[lemmatizer.lemmatize(word) for word in words]

    return ' '.join(lemmatized_words)

df['clean_review']=df['clean_review'].apply(lematize_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...


### Perform Sentiment Analysis

### using Textblob

In [28]:
from textblob import TextBlob

def analyze_sentiment(reviews):
    sentiment_scores=[]
    for review in reviews:
        blob=TextBlob(review)
        sentiment_score=blob.sentiment.polarity
        sentiment_scores.append(sentiment_score)
    return sentiment_scores

df['sentiment_score']=analyze_sentiment(df['clean_review'])

In [29]:
from textblob import TextBlob

def analyze_sentiment(reviews):
    sentiment_scores = []
    for review in reviews:
        blob = TextBlob(review)
        sentiment_score = blob.sentiment.polarity
        return "Positive" if sentiment_score > 0 else "Negative" if sentiment_score < 0 else "Neutral"

df['sentiment'] = df['clean_review'].apply(analyze_sentiment)

### Evaluation performance

In [30]:
from sklearn.metrics import accuracy_score, classification_report

print(classification_report(df['sentiment'],df['sentiment']))
print(accuracy_score(df['sentiment'],df['sentiment']))

              precision    recall  f1-score   support

     Neutral       1.00      1.00      1.00     50000

    accuracy                           1.00     50000
   macro avg       1.00      1.00      1.00     50000
weighted avg       1.00      1.00      1.00     50000

1.0
